newgrp docker

./start-simulation.sh

## Stop the Receiver

In [114]:
import socket

PACKET_SIZE = 1024
SEQ_ID_SIZE = 4
MESSAGE_SIZE = PACKET_SIZE - SEQ_ID_SIZE
Receiver = ('localhost', 5001)
def sendFinack(sender):
    finack = int.to_bytes(0, SEQ_ID_SIZE, byteorder='big', signed=True) + b'==FINACK=='
    sender.sendto(finack, Receiver)
    sender.close()
sender = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sender.bind(("localhost", 5002))
sender.settimeout(1)

sendFinack(sender)

## Stop and Wait

In [116]:
# Code has been sampled from Muhammad Haroon's code, with modifications.
# Original code: https://github.com/Haroon96/ecs152a-fall-2023/blob/main/week7/docker/sender.py

import socket
from time import time 

# total packet size
PACKET_SIZE = 1024
# bytes reserved for sequence id
SEQ_ID_SIZE = 4
# bytes available for message
MESSAGE_SIZE = PACKET_SIZE - SEQ_ID_SIZE

# read data
with open('./docker/file.mp3', 'rb') as f:
    data = f.read()
 
total_packet_delay = 0
packetCount = 0

# create a udp socket
start_throughput = time()
with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as udp_socket:

    # bind the socket to a OS port
    udp_socket.bind(("0.0.0.0", 5000))

    timeoutDuration = 1
    
    # start sending data from 0th sequence
    seq_id = 0
    sent_empty = False
    # Run a timer for throughput
    while True:
        
        udp_socket.settimeout(1)
        
        # construct message
        # sequence id of length SEQ_ID_SIZE + message of remaining PACKET_SIZE - SEQ_ID_SIZE bytes
        message = int.to_bytes(seq_id, SEQ_ID_SIZE, byteorder='big', signed=True) + data[seq_id : seq_id + MESSAGE_SIZE]

        # constructs the empty packet if we have sent all previous data
        if seq_id > len(data) and not sent_empty:
            message = int.to_bytes(len(data), SEQ_ID_SIZE, byteorder='big', signed=True)
            sent_empty = True
            print("is making")
            

        # send message
        udp_socket.sendto(message, ('localhost', 5001))
        packet_delay = time()
        packetCount += 1
        
        # wait for acknowledgement
        ack_id = 0
        while True:
            try:
                # wait for ack
                ack, _ = udp_socket.recvfrom(PACKET_SIZE)
                total_packet_delay += time() - packet_delay

                # extract ack id
                ack_id = int.from_bytes(ack[:SEQ_ID_SIZE], byteorder='big')
                
                print(ack_id, ack[SEQ_ID_SIZE:])

                if ack_id != len(data) or not sent_empty:
                    break
                
            except socket.timeout:
                # no ack received, resend unacked message
                timeoutDuration += timeoutDuration
                udp_socket.settimeout(timeoutDuration)
                udp_socket.sendto(message, ('localhost', 5001))
          
        if ack_id == len(data) + 3:
            break
    
        # move sequence id forward
        seq_id += MESSAGE_SIZE      

    # Run the time until the last packet from the file, and NOT the final closing message. 
    end_throughput = time()
    
    # get throughput
    throughput  = len(data) / (end_throughput - start_throughput)

    # get average packet delay
    avg_packet_delay = total_packet_delay / packetCount

    # get performance metric (throughput/average per packet delay)
    performance_metric = throughput / avg_packet_delay

    print(f'{round(throughput, 2)}, {round(avg_packet_delay, 2)}, {round(performance_metric, 2)}')
    
    # send final closing message
    finack = int.to_bytes(0, SEQ_ID_SIZE, byteorder='big', signed=True) + b'==FINACK=='
    udp_socket.sendto(finack, ('localhost', 5001))
    
    # close the connection
    udp_socket.close()


1020 b'ack'
2040 b'ack'
3060 b'ack'
4080 b'ack'
5100 b'ack'
6120 b'ack'
7140 b'ack'
8160 b'ack'
9180 b'ack'
10200 b'ack'
11220 b'ack'
12240 b'ack'
13260 b'ack'
14280 b'ack'
15300 b'ack'
16320 b'ack'
17340 b'ack'
18360 b'ack'
19380 b'ack'
20400 b'ack'
21420 b'ack'
22440 b'ack'
23460 b'ack'
24480 b'ack'
25500 b'ack'
26520 b'ack'
27540 b'ack'
28560 b'ack'
29580 b'ack'
30600 b'ack'
31620 b'ack'
32640 b'ack'
33660 b'ack'
34680 b'ack'
35700 b'ack'
36720 b'ack'
37740 b'ack'
38760 b'ack'
39780 b'ack'
40800 b'ack'
41820 b'ack'
42840 b'ack'
43860 b'ack'
44880 b'ack'
45900 b'ack'
46920 b'ack'
47940 b'ack'
48960 b'ack'
49980 b'ack'
51000 b'ack'
52020 b'ack'
53040 b'ack'
54060 b'ack'
55080 b'ack'
56100 b'ack'
57120 b'ack'
58140 b'ack'
59160 b'ack'
60180 b'ack'
61200 b'ack'
62220 b'ack'
63240 b'ack'
64260 b'ack'
65280 b'ack'
66300 b'ack'
67320 b'ack'
68340 b'ack'
69360 b'ack'
70380 b'ack'
71400 b'ack'
72420 b'ack'
73440 b'ack'
74460 b'ack'
75480 b'ack'
76500 b'ack'
77520 b'ack'
78540 b'ack'
79560 b'

Output: 9898.89, 0.1, 97115.41 

# Sliding Window

In [115]:
# Code has been sampled from Muhammad Haroon's code, with modifications.
# Original code: https://github.com/Haroon96/ecs152a-fall-2023/blob/main/week7/docker/sender.py

import socket
from time import time 

# total packet size
PACKET_SIZE = 1024
# bytes reserved for sequence id
SEQ_ID_SIZE = 4
# bytes available for message
MESSAGE_SIZE = PACKET_SIZE - SEQ_ID_SIZE
# total packets to send
WINDOW_SIZE = 100

# read data
with open('./docker/file.mp3', 'rb') as f:
    data = f.read()
 
total_packet_delay = 0
packetCount = 0
 
# create a udp socket
start_throughput = time()
with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as udp_socket:

    # bind the socket to a OS port
    udp_socket.bind(("0.0.0.0", 5000))

    timeoutDuration = 1

    # start sending data from 0th sequence
    seq_id = 0
    windowSpace = WINDOW_SIZE

    acks = {}
    packet_start_times = {}
    packet_end_times = {}
    udp_socket.settimeout(1)

    sent_empty = False

    ack_id = 0
    while True:
        for _ in range(windowSpace):
            message = int.to_bytes(seq_id, SEQ_ID_SIZE, byteorder='big', signed=True) + data[seq_id: seq_id + MESSAGE_SIZE]
            
            # constructs the empty packet if we have sent all previous data
            if seq_id > len(data) and not sent_empty:
                message = int.to_bytes(len(data), SEQ_ID_SIZE, byteorder='big', signed=True)
                sent_empty = True
            
            acks[seq_id] = False
            
            if seq_id not in packet_start_times:
                packet_start_times[seq_id] = time()

            udp_socket.sendto(message, ('localhost', 5001))

            seq_id += MESSAGE_SIZE
            windowSpace -= 1

            if len(message) == 0:
                break
            
        try:
            if ack_id in packet_start_times and time() - packet_start_times[ack_id] >= timeoutDuration:
                print()
                print("timeout resend ack_id: ", ack_id)
                message = int.to_bytes(ack_id, SEQ_ID_SIZE, byteorder='big', signed=True) + data[ack_id: ack_id + MESSAGE_SIZE]

                udp_socket.sendto(message, ('localhost', 5001))
                packet_start_times[ack_id] = time()
                
            ack, _ = udp_socket.recvfrom(PACKET_SIZE)
            
            ack_id = int.from_bytes(ack[:SEQ_ID_SIZE], byteorder='big')
            ack_message = ack[SEQ_ID_SIZE:]
            
            if ack_message == b'fin':
                break

            # update acks below cumulative ack
            for a in acks:
                if a < ack_id and acks[a] != True:
                    acks[a] = True
                    windowSpace += 1
                    if a not in packet_end_times:
                            packet_end_times[a] = time()

        except socket.timeout:
            pass
     
    
    # Run the time until the last packet from the file, and NOT the final closing message. 
    end_throughput = time()

    # send final closing message
    finack = int.to_bytes(0, SEQ_ID_SIZE, byteorder='big', signed=True) + b'==FINACK=='
    udp_socket.sendto(finack, ('localhost', 5001))   
    
    # get throughput
    throughput  = len(data) / (end_throughput - start_throughput)

    # get average packet delay
    avg_delay = 0
    for k in packet_end_times.keys():
        packet_delay = packet_end_times[k] - packet_start_times[k]
        avg_delay += packet_delay

    avg_delay /= len(packet_end_times.keys())

    # get performance metric (throughput/average per packet delay)
    performance_metric = throughput / avg_delay

    print(f'{round(throughput, 2)}, {round(avg_delay, 2)}, {round(performance_metric, 2)}')

    # close the connection
    udp_socket.close()
    

0 b'ack'
timeout resend ack_id:  0
1020 b'ack'
timeout resend ack_id:  1020
2040 b'ack'
timeout resend ack_id:  2040
3060 b'ack'
timeout resend ack_id:  3060
4080 b'ack'
timeout resend ack_id:  4080
5100 b'ack'
timeout resend ack_id:  5100
6120 b'ack'
timeout resend ack_id:  6120
7140 b'ack'
timeout resend ack_id:  7140
168300 b'ack'
timeout resend ack_id:  168300
169320 b'ack'
timeout resend ack_id:  169320
170340 b'ack'
timeout resend ack_id:  170340
171360 b'ack'
timeout resend ack_id:  171360
172380 b'ack'
timeout resend ack_id:  172380
173400 b'ack'
timeout resend ack_id:  173400
174420 b'ack'
timeout resend ack_id:  174420
175440 b'ack'
timeout resend ack_id:  175440
176460 b'ack'
timeout resend ack_id:  176460
177480 b'ack'
timeout resend ack_id:  177480
178500 b'ack'
timeout resend ack_id:  178500
179520 b'ack'
timeout resend ack_id:  179520
180540 b'ack'
timeout resend ack_id:  180540
181560 b'ack'
timeout resend ack_id:  181560
182580 b'ack'
timeout resend ack_id:  182580
183

51266.76, 0.29, 174616.9 1.44s